# LIONS Analysis

Our project is to analyze federal cases.

In [1]:
from lions.parse import *
import pandas as pd
import numpy as np

In [3]:
case_sql, cases = gs_case(columns=['ID', 'DISTRICT', 'LEAD_CHARGE'])
hist_sql, hist = gs_court_hist(columns=['CASEID', 'COURT', 'APPEAL_TYPE', 'SENT_APPEAL', \
                                        'DISPOSITION', 'DISP_REASON1', 'DISP_REASON2', 'DISP_REASON3'])
court_judge_sql, court_judge = gs_court_judge(columns=['CASEID', 'JUDGEID', 'DECISION'])
judge_sql, judge = gs_judge()

In [5]:
#cases = cases[:100000]
#hist = hist[:50000]
#court_judge = court_judge[:50000]

cases_df = pd.DataFrame(cases, columns = cases[0].keys())
del cases

hist_df = pd.DataFrame(hist, columns = hist[0].keys())
del hist

court_judge_df = pd.DataFrame(court_judge, columns = court_judge[0].keys())
del court_judge

judge_df = pd.DataFrame(judge, columns = judge[0].keys())
del judge

print(len(cases_df))
print(len(hist_df))

6041748
8262351


In [ ]:
cases_hist_df = pd.merge(cases_df, hist_df, left_on = 'ID', right_on = 'CASEID', how = 'inner')
del cases_df
del hist_df
print("Done merging Cases DF and Hist DF")

cases_hist_court_df = pd.merge(cases_hist_df, court_judge_df, left_on = 'CASEID', right_on = 'CASEID', how = 'inner')
del cases_hist_df
del court_judge_df
print("Done merging Cases Hist DF and Court Judge DF")

cases_hist_court_judge_df = pd.merge(cases_hist_court_df, judge_df, left_on = ['JUDGEID','DISTRICT'], right_on = ['ID','DISTRICT'], how = 'inner')
del cases_hist_court_df
del judge_df
print("Done merging Cases Hist Court DF and Judge DF")

withdisp_df = cases_hist_court_judge_df[cases_hist_court_judge_df.DISPOSITION != '']
del case_hist_court_judge_df

print(len(withdisp_df))
withdisp_reason_df = withdisp_df[withdisp_df.DISP_REASON1 != '']
print(len(withdisp_reason_df))
withoutdisp_reason_df = withdisp_df[withdisp_df.DISP_REASON1 == '']
print(len(withoutdisp_reason_df))
del withdisp_df

In [5]:
columns_to_drop = ['ID_x','NAME','RECVD_DATE','SECURITY','PRIORITY', 'TYPE_x', 'US_ROLE_x','LIT_RESP', \
                   'LIT_TRACK','WEIGHT','BRANCH','GRAND_JURY_NUM', 'UNIT', 'DCMNS_NUMBER', 'TRIBE','RESERVATION', \
                   'SPEC_PROJ', 'VIC_WIT', 'ADR_MODE', 'COLLECT_IND','OFFENSE_FROM', 'OFFENSE_TO', 'PHYSICAL_LOC', \
                   'STORE_NUM','SYS_INIT_DATE', 'ACCESS_DATE', 'STATUS','CLOSE_DATE', 'DEST_DATE', 'PERMANENT', \
                   'RELATED_FLU_FLAG', 'QUI_TAM','CREATE_DATE_x', 'CREATE_USER_x', 'UPDATE_DATE_x', 'UPDATE_USER_x', \
                   'DISTRICT_y', 'ID_y', 'LOCATION', 'US_ROLE_y','COURT_NUMBER', 'FILING_DATE', 'SERVICE_DATE', \
                   'NOAP_DATE','DISP_DATE','DISP_REASON1', 'DISP_REASON2', 'DISP_REASON3', 'SYS_DISP_DATE','SYS_FILING_DATE', \
                   'CREATE_DATE_y', 'CREATE_USER_y', 'UPDATE_DATE_y','UPDATE_USER_y', 'DISTRICT', 'CRTHISID', 'ID_x', 'JUDGEID', \
                   'START_DATE', 'END_DATE', 'CREATE_DATE_x', 'CREATE_USER_x','UPDATE_DATE_x', 'UPDATE_USER_x', 'ID_y', 'INITIALS', \
                   'INIT_STAT','LAST_NAME', 'FIRST_NAME', 'COURT_ROOM', 'TYPE_y', 'CREATE_DATE_y','CREATE_USER_y', 'UPDATE_DATE_y','UPDATE_USER_y', \
                   'CIVIL_POTEN','CASE_RESTRICTED','CRIMINAL_POTEN','TOT_VICTIMS','DECISION','TRIAL_DAYS']

disp_reasons = ['VACA','VARM','RDAP','RDRR','REVA','REVR','OFPO','GWDA','DEPO','LECI','WKEV','WTPR']


misconduct = []
disp1 = withdisp_reason_df['DISP_REASON1'].tolist()

for i in range(len(disp1)):
    if disp1[i] in disp_reasons: 
        misconduct.append(1)
    else:
        misconduct.append(0)

withdisp_reason_df.set_index('CASEID', inplace=True)
withoutdisp_reason_df.set_index('CASEID', inplace=True)

df_classify = withdisp_reason_df.drop(columns_to_drop, axis = 1)
df_to_classify = withoutdisp_reason_df.drop(columns_to_drop, axis=1)

In [6]:
df_district = pd.get_dummies(df_classify['DISTRICT_x'], prefix = 'District')
df_class = pd.get_dummies(df_classify['CLASS'], prefix = 'Class')
df_charge = pd.get_dummies(df_classify['LEAD_CHARGE'], prefix = 'Charge')
df_court = pd.get_dummies(df_classify['COURT'], prefix = 'Court')
df_appeal = pd.get_dummies(df_classify['APPEAL_TYPE'], prefix = 'Appeal')
df_sent = pd.get_dummies(df_classify['SENT_APPEAL'], prefix = 'Sent')
df_disposition = pd.get_dummies(df_classify['DISPOSITION'], prefix = 'Disposition')

df_new = pd.concat([df_classify, df_district, df_class, df_charge, df_court, df_appeal, df_sent, df_disposition], axis=1)
df_new = df_new.drop(['DISTRICT_x','CLASS','LEAD_CHARGE', 'COURT', 'APPEAL_TYPE','SENT_APPEAL','DISPOSITION'], axis=1)

df_district = pd.get_dummies(df_to_classify['DISTRICT_x'], prefix = 'District')
df_class = pd.get_dummies(df_to_classify['CLASS'], prefix = 'Class')
df_charge = pd.get_dummies(df_to_classify['LEAD_CHARGE'], prefix = 'Charge')
df_court = pd.get_dummies(df_to_classify['COURT'], prefix = 'Court')
df_appeal = pd.get_dummies(df_to_classify['APPEAL_TYPE'], prefix = 'Appeal')
df_sent = pd.get_dummies(df_to_classify['SENT_APPEAL'], prefix = 'Sent')
df_disposition = pd.get_dummies(df_to_classify['DISPOSITION'], prefix = 'Disposition')

df_new_to_classify = pd.concat([df_to_classify, df_district, df_class, df_charge, df_court, df_appeal, df_sent, df_disposition], axis=1)

df_new_to_classify = df_new_to_classify.drop(['DISTRICT_x','CLASS','LEAD_CHARGE', 'COURT', 'APPEAL_TYPE','SENT_APPEAL','DISPOSITION'], axis=1)


common_cols = list(set(df_new).intersection(df_new_to_classify))

df_new = df_new[common_cols]
df_new_to_classify = df_new_to_classify[common_cols]
df_new

,Court_CA,Charge_18 :02252a4B,Charge_18 :01708,Charge_26 :07212,Court_DC,Charge_18 :00876,Charge_18 :00924E,Appeal_O,Charge_18 :01005,Charge_47 :00553a,...,Charge_21 :00844a,Disposition_DA,Sent_Y,Charge_18 :02252,Disposition_DP,Class_R,Charge_21 :00841(a),Charge_18 :00032,Charge_18 :00924c,Charge_18 :01382
CASEID,,,,,,,,,,,,,,,,,,,,,
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
#generate our train and test dataset
m = 3000

df_to_be_classified = df_new_to_classify

df_train = df_new.iloc[:m]
df_test = df_new.iloc[m:]
list_train = misconduct[:m]
list_test = misconduct[m:]

df_train

,Court_CA,Charge_18 :02252a4B,Charge_18 :01708,Charge_26 :07212,Court_DC,Charge_18 :00876,Charge_18 :00924E,Appeal_O,Charge_18 :01005,Charge_47 :00553a,...,Charge_21 :00844a,Disposition_DA,Sent_Y,Charge_18 :02252,Disposition_DP,Class_R,Charge_21 :00841(a),Charge_18 :00032,Charge_18 :00924c,Charge_18 :01382
CASEID,,,,,,,,,,,,,,,,,,,,,
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1989R00022,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
from sklearn.neighbors import KNeighborsClassifier
import sklearn.model_selection as cross_validation

#build our knn model and print the accuracy score
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(df_train, list_train)

#print the accuracy score
print('KNeighbor accuracy on test data: ', knn.score(df_test, list_test))
print('KNeighbor accuracy on training data: ', knn.score(df_train, list_train))

#accuracy score from crossvalidation
scores = cross_validation.cross_val_score(knn, df_new, misconduct, cv=5)
print("KNN Accuracy from crossvalidation:: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

KNeighbor accuracy on test data:  0.958860892256316
KNeighbor accuracy on training data:  0.968
KNN Accuracy from crossvalidation:: 0.96 (+/- 0.01)


In [9]:
from sklearn import tree

#build our decision trees model
dtc = tree.DecisionTreeClassifier()
dtc.fit(df_train, list_train)

#print the accuracy score
print('DT accuracy on test data: ', dtc.score(df_test, list_test))
print('DT accuracy on training data: ', dtc.score(df_train, list_train))

#accuracy score from crossvalidation
scores = cross_validation.cross_val_score(dtc, df_new, misconduct, cv=5)
print("DT Accuracy from crossvalidation: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

DT accuracy on test data:  0.9581057383307742
DT accuracy on training data:  0.9856666666666667
DT Accuracy from crossvalidation: 0.96 (+/- 0.01)


In [10]:
from sklearn.linear_model import LogisticRegression

#build our logistic regression model
lgr = LogisticRegression(solver='lbfgs', multi_class='multinomial')
lgr.fit(df_train, list_train)

#print the accuracy score
print("Accuracy of logistic regression test set:", lgr.score(df_test, list_test))
print("Accuracy of logistic regression train set:", lgr.score(df_train, list_train))

#accuracy score from crossvalidation
scores = cross_validation.cross_val_score(lgr, df_new, misconduct, cv=5)
print("Logistic Regression Accuracy from crossvalidation: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy of logistic regression test set: 0.9597754675661389
Accuracy of logistic regression train set: 0.978
Logistic Regression Accuracy from crossvalidation: 0.96 (+/- 0.01)


In [11]:
#Apply the classifier to the dataset withoutdisp_reason_df

df_to_be_classified

,Court_CA,Charge_18 :02252a4B,Charge_18 :01708,Charge_26 :07212,Court_DC,Charge_18 :00876,Charge_18 :00924E,Appeal_O,Charge_18 :01005,Charge_47 :00553a,...,Charge_21 :00844a,Disposition_DA,Sent_Y,Charge_18 :02252,Disposition_DP,Class_R,Charge_21 :00841(a),Charge_18 :00032,Charge_18 :00924c,Charge_18 :01382
CASEID,,,,,,,,,,,,,,,,,,,,,
1983V01272,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1986V01040,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1989V00145,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990V00059,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990V00107,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990V00131,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990V00235,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990V00252,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1991V00179,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_prediction_knn = lgr.predict(df_to_be_classified)

df_results = df_to_be_classified.copy()
df_results['misconduct'] = df_prediction_knn
df_misconduct = df_results[df_results['misconduct'] == 1]
df_misconduct

,Court_CA,Charge_18 :02252a4B,Charge_18 :01708,Charge_26 :07212,Court_DC,Charge_18 :00876,Charge_18 :00924E,Appeal_O,Charge_18 :01005,Charge_47 :00553a,...,Disposition_DA,Sent_Y,Charge_18 :02252,Disposition_DP,Class_R,Charge_21 :00841(a),Charge_18 :00032,Charge_18 :00924c,Charge_18 :01382,misconduct
CASEID,,,,,,,,,,,,,,,,,,,,,
1993V00230,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1993V00230,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1993V00230,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1995V00155,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1995V00155,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1995V00236,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1995V00236,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1995V00236,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1999V00139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
col_name = 'District'
df_misconduct_new = df_misconduct.copy()
df_misconduct_new.columns = df_misconduct_new.columns.str.split('_', expand=True)
df_misconduct_new = (df_misconduct_new.replace(0, np.nan)
                     .stack().reset_index()
                     .drop([col_name, 'level_0'], 1)
                     .rename(columns={'level_1':col_name})
                    )

df_misconduct_new

KeyError: "['level_0'] not found in axis"